In [ ]:
# Import the notebook-specific login utility from Hugging Face
from huggingface_hub import notebook_login

# This will prompt the user to enter their Hugging Face access token to enable access to private models and datasets
notebook_login()

In [ ]:
# Upgrade or install bitsandbytes - a lightweight CUDA library for 8-bit optimizers and quantization
!pip install -U bitsandbytes

# Install libraries used for:
# - faiss-cpu: fast vector similarity search
# - sentence-transformers: pretrained models for sentence embeddings
# - transformers: Hugging Face library for transformer-based models
!pip install faiss-cpu sentence-transformers transformers


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 MB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 65.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 36.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 67.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstallin

In [ ]:
# This block sets up the environment for running a quantized large language model (Zephyr-7B) from Hugging Face.
# It installs core libraries (Pandas, NumPy, FAISS, SentenceTransformers, Transformers), 
# initializes the device (GPU if available), sets up 4-bit quantization using BitsAndBytesConfig,
# and loads the tokenizer and model into memory using the specified quantization settings.

import pandas as pd
import numpy as np
import faiss
from sentence_transformers import SentenceTransformer, util
from transformers import pipeline

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "HuggingFaceH4/zephyr-7b-beta"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

from transformers import BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    quantization_config=bnb_config,
)

model.to(device)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

...

model-00003-of-00008.safetensors:   0%|          | 0.00/1.98G [00:00<?, ?B/s]

model-00007-of-00008.safetensors:   0%|          | 0.00/1.98G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32000, 4096, padding_idx=2)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
        )
        (mlp): MistralMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): MistralRMSNorm((4096,), eps=1e-05)
      )
    )
 

In [4]:
## Slave 1

In [ ]:
# This block loads housing data from a CSV, transforms each row into a structured text prompt,
# generates sentence embeddings using a pretrained MiniLM model, indexes them with FAISS for similarity search,
# and defines a function to retrieve the most relevant entries based on a user query.

# Step 1: Load your dataset
df = pd.read_csv("Housing_Data_with_Availability.csv")

# Convert each row of the dataset into a readable prompt-like text block
def row_to_text(row):
    details = ""
    for col in df.columns:
        details += f"{col}: {row[col]}\n"
    return details

# Create a list of prompt-style strings for each housing entry
texts = df.apply(row_to_text, axis=1).tolist()

# Step 3: Generate sentence embeddings for the housing descriptions
embedder = SentenceTransformer("all-MiniLM-L6-v2")
embeddings = embedder.encode(texts, convert_to_tensor=False)

# Step 4: Create and populate a FAISS index for similarity search
embedding_dim = embeddings[0].shape[0]
index = faiss.IndexFlatL2(embedding_dim)
index.add(np.array(embeddings))

# Function to fetch top_k most relevant housing entries for a given query
def get_prompt(question, top_k=3):
    q_embedding = embedder.encode(question, convert_to_tensor=False)
    D, I = index.search(np.array([q_embedding]), top_k)
    context = "\n\n".join([texts[i] for i in I[0]])
    return context


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# This function generates a response to a user query using a quantized LLM and a FAISS-retrieved housing context.
# It constructs a structured Q&A prompt with the provided context, feeds it into the model,
# and returns a clean, point-based answer relevant to housing features.

def generate_answer(question, context, max_length=500):
    """
    Generates an answer using a quantized large language model (Zephyr-7B) based on the given question and FAISS-retrieved housing context.
    The model is instructed to respond with short, point-wise answers grounded strictly in the provided data.
    """

    # Format input prompt with assistant persona, context, and user question
    prompt = f"""You are a helpful real estate assistant with access to detailed data about housing prices, rent, availability, and utilities across multiple cities and neighborhoods in the United States.

      Use the following context to answer the user's question. Be precise, refer only to data in the context, and include specific values when available. Give all the answer in short points, do not create a para answer

      Context:
      {context}

      Question:
      {question}

      Answer:
      """

    # Tokenize input prompt and move tensors to GPU or CPU based on availability
    inputs = tokenizer(prompt, return_tensors="pt").to(device)

    # Generate output from the language model without computing gradients
    with torch.no_grad():
        outputs = model.generate(
            input_ids=inputs["input_ids"].to(device),
            attention_mask=inputs["attention_mask"].to(device),
            max_new_tokens=max_length,
            temperature=0.7,
            pad_token_id=tokenizer.eos_token_id,
            top_k=50,
            top_p=0.9
        )

    # Decode the model's output and extract only the final answer part
    response = tokenizer.decode(outputs[0].cpu(), skip_special_tokens=True)
    answer = response.split("Answer:")[-1].strip()

    return answer


In [ ]:
# This function handles end-to-end housing queries from users.
# It retrieves the most relevant entries from the dataset using semantic similarity (FAISS + MiniLM),
# and generates a precise, point-based answer using the LLM grounded in those entries.

def housing_pred(question):
    # Retrieve similar housing entries using semantic search
    context = get_prompt(question)

    # Generate an answer grounded in the retrieved context
    answer = generate_answer(question, context)

    return answer


In [9]:
## Slave 2

In [ ]:
# This block prepares hate crime data for semantic search using sentence embeddings and FAISS.
# It loads a CSV of hate crimes, converts each row to a descriptive text format,
# embeds them using MiniLM, builds a FAISS index for similarity search,
# and defines a function to retrieve top-k relevant rows for a given question.

import pandas as pd
import numpy as np
import faiss
from sentence_transformers import SentenceTransformer, util
from transformers import pipeline

# Load hate crime dataset
df = pd.read_csv("/content/HateCrime_By_Motivation_Expanded_FINAL.csv")

# Convert each row into a detailed text string for embedding
def row_to_text_crime(row):
    details = ""
    for col in df.columns:
        details += f"{col}: {row[col]}\n"
    return details

# Generate prompt-style strings from the dataset
texts = df.apply(row_to_text_crime, axis=1).tolist()

# Create embeddings using a lightweight SentenceTransformer model
embedder = SentenceTransformer("all-MiniLM-L6-v2")
embeddings = embedder.encode(texts, convert_to_tensor=False)

# Build a FAISS index for fast nearest neighbor search
embedding_dim = embeddings[0].shape[0]
index = faiss.IndexFlatL2(embedding_dim)
index.add(np.array(embeddings))

# Function to retrieve the most relevant crime entries given a user query
def get_prompt_crime(question, top_k=3):
    q_embedding = embedder.encode(question, convert_to_tensor=False)
    D, I = index.search(np.array([q_embedding]), top_k)
    context = "\n\n".join([texts[i] for i in I[0]])
    return context


In [ ]:
# This function generates a crime-related answer to a user query based on FAISS-retrieved hate crime data.
# It builds a structured prompt instructing the LLM to evaluate safety, crime level, and types of crimes using context.
# The LLM is expected to return short bullet points with realistic interpretations and data-based reasoning.

def generate_answer_crime(question, context, max_length=500):
    """
    Generates an answer using a quantized large language model based on the user's safety-related question and relevant hate crime context.
    The model responds with short, specific, data-driven bullet points summarizing crime safety and trends.
    """

    prompt = f"""You are a helpful real estate assistant with access to detailed crime statistics and housing data across multiple cities and neighborhoods in the United States.

      Use the following context to assess and answer the user's question. Be clear, specific, and data-driven. When referring to crime, analyze whether the city is considered *safe or unsafe*, describe the general crime rate (e.g., low, moderate, high), and mention the most common types of crimes in that location. Back up your conclusion with at least one realistic example-style sentence to illustrate the city's situation (e.g., “It's not uncommon for locals to avoid walking alone in certain areas after dark due to frequent theft reports.”)

      Keep your answers in **short bullet points** — no paragraphs.

      Context:
      {context}

      Question:
      {question}

      Answer:

      """

    # Tokenize and prepare model input on the correct device
    inputs = tokenizer(prompt, return_tensors="pt").to(device)

    with torch.no_grad():
        outputs = model.generate(
            input_ids=inputs["input_ids"].to(device),
            attention_mask=inputs["attention_mask"].to(device),
            max_new_tokens=max_length,
            temperature=0.7,
            pad_token_id=tokenizer.eos_token_id,
            top_k=50,
            top_p=0.9
        )

    # Decode model output and extract answer section
    response = tokenizer.decode(outputs[0].cpu(), skip_special_tokens=True)
    answer = response.split("Answer:")[-1].strip()

    return answer


In [ ]:
# This function serves as the main interface for handling safety-related user queries.
# It retrieves the most relevant crime records using semantic search,
# then generates a summary of the crime situation using an LLM, with realistic and data-backed insights.

def crime_pred(question):
    # Get the most relevant crime entries using FAISS semantic search
    context = get_prompt_crime(question)

    # Generate a response using the crime-focused prompt template
    answer = generate_answer_crime(question, context)

    return answer

# Example usage (commented out for production use):
# question = "How is crime rate in New York (Buffalo)"
# context = get_prompt_crime(question)
# answer = generate_answer_crime(question, context)
# print("Answer:", answer)

In [ ]:
# This function is the central multi-agent query handler. It parses the user's question into crime/housing parts,
# runs them through their respective pipelines, and finally rewrites the result in a friendly, chat-style summary.
# It uses prompt-based inference to decide what to split, retrieve, and summarize using an LLM.

import re

chat_history = ""  # Tracks conversation context for vague reference resolution

def generate_answer_main(content, chat_history, max_length=500):
    """
    Main entry point that:
    1. Parses the user's query into sub-questions about crime and housing.
    2. Routes them through specialized retrieval + response agents.
    3. Combines the raw answers into a natural, conversational final response.
    """

    # Prompt to split the query into crime and housing components based on intent and chat history
    splitter_prompt = f"""
You are a question analyzer. Your task is to determine whether the user's current question includes topics related to crime and/or housing.

If both topics are present, split the question into two clear sub-questions. If only one is relevant, keep the other key as null.

Respond strictly using this JSON format:
{{
  "crime": "<crime-related sub-question or null>",
  "House_price": "<housing-related sub-question or null>"
}}

Definitions:
- Crime-related: questions about safety, crime rate, violent/property crime, gun violence, police response, etc.
- Housing-related: questions about rent, cost of living, affordability, availability, utility costs, etc.

Use the following context to improve your understanding of the user’s intent:
---
Chat History Summary:
{chat_history}
---

Only use this summary to **disambiguate vague references** like "that area" or "that city" and clarify the user's preferences or past concerns.

--- Now analyze the current user question below ---

User Question: {content}

Your task: Return only the JSON dictionary with the appropriate sub-questions.
"""

    # Tokenize and generate the split query response
    inputs = tokenizer(splitter_prompt, return_tensors="pt").to(device)

    with torch.no_grad():
        outputs = model.generate(
            input_ids=inputs["input_ids"].to(device),
            attention_mask=inputs["attention_mask"].to(device),
            max_new_tokens=max_length,
            temperature=0.7,
            pad_token_id=tokenizer.eos_token_id,
            top_k=50,
            top_p=0.9
        )

    # Decode and extract the split question JSON
    response = tokenizer.decode(outputs[0].cpu(), skip_special_tokens=True)
    response_str = response.split("--- Now output ONLY the JSON below ---")[-1].strip()

    # Extract sub-questions using regex
    crime_match = re.search(r'"crime"\s*:\s*"([^"]*)",', response_str)
    housing_match = re.search(r'"House_price"\s*:\s*"([^"]*)"', response_str)

    crime_question = crime_match.group(1) if crime_match else None
    house_price_question = housing_match.group(1) if housing_match else None

    # Generate individual agent responses
    answer = ""
    if crime_question:
        answer += crime_pred(crime_question)
    if house_price_question:
        answer += housing_pred(house_price_question)

    # Prompt to rewrite the result in a friendly, human tone
    combine_prompt = f"""
    You are a friendly and conversational summarization assistant. Your job is to take a given **user question** and its corresponding **answer**, and rewrite the answer to feel more natural, chat-like, and human — while still accurately and fully answering the original question.

      Be sure to:
      - Keep the tone engaging and conversational (as if speaking to a friend).
      - Include friendly expressions where appropriate (e.g., "Absolutely!", "Sure!", "Honestly," etc.).
      - Make sure the rewritten summary directly addresses the user's question and preserves all key information.

      Format:
      Question: <user's question>
      Original Answer: <raw model or data answer>
      Chat-style Summary: <your friendly, conversational rewritten version>

      Question: {content}
      Original Answer: {answer}

      Chat-style Summary:
    """

    # Tokenize and generate the chat-style summary
    inputs = tokenizer(combine_prompt, return_tensors="pt").to(device)

    with torch.no_grad():
        outputs = model.generate(
            input_ids=inputs["input_ids"].to(device),
            attention_mask=inputs["attention_mask"].to(device),
            max_new_tokens=max_length,
            temperature=0.7,
            pad_token_id=tokenizer.eos_token_id,
            top_k=50,
            top_p=0.9
        )

    # Decode final friendly summary
    response = tokenizer.decode(outputs[0].cpu(), skip_special_tokens=True)
    answer = response.split("Chat-style Summary:")[-1].strip()

    return answer


In [ ]:
# This module tracks recent user-bot interactions and generates a summarized context string.
# It's used to help the LLM understand prior preferences or constraints, improving multi-turn dialogue accuracy.

conversation_history = []

def update_history(user_input, bot_response):
    """
    Updates the chat history with the latest user and bot messages.
    Keeps only the last 2 user-bot pairs (4 lines total) to limit context length.
    """
    conversation_history.append(f"User: {user_input}")
    conversation_history.append(f"Bot: {bot_response}")

    # Keep only the last 4 turns (2 user + 2 bot)
    if len(conversation_history) > 4:
        conversation_history[:] = conversation_history[-4:]

def summarize_conversation(history, max_length=500):
    """
    Generates a clean, structured summary of the conversation history.
    This helps downstream models maintain context, user preferences, and intent.
    The summary is engineered for machine-readability and grounded in relevant details only.
    """

    prompt = f"""
    You are a smart assistant helping to generate a context summary for a language model.

    Your goal is to extract a clean and structured summary of the user's goals, preferences, tone, and prior questions.
    This summary will be sent to another LLM so that it can give better, more contextual answers in future turns.

    Please do the following:
    1. Remove filler words, small talk, or vague expressions.
    2. Focus only on relevant facts, intents, location names, preferences (like low crime or affordable housing).
    3. Restructure key points into a compact paragraph that feels engineered for machine-readability.
    4. You may simplify, reorder, or reformat information to improve interpretability (light feature engineering).
    5. Use consistent phrasing and entity labels where possible.

    Conversation history:
    {" ".join(history)}

    Summary:
    """

    # Tokenize and generate structured summary
    inputs = tokenizer(prompt, return_tensors="pt").to(device)

    with torch.no_grad():
        outputs = model.generate(
            input_ids=inputs["input_ids"],
            attention_mask=inputs["attention_mask"],
            max_new_tokens=max_length,
            temperature=0.7,
            pad_token_id=tokenizer.eos_token_id,
            top_k=50,
            top_p=0.9
        )

    summary = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return summary.split("Summary:")[-1].strip()


In [18]:
question = "Give me a city with less crime and less housing price in California"

answer = generate_answer_main(question, chat_history)

update_history(question, answer)
print("Answer:", answer)

chat_history = summarize_conversation(conversation_history)


Answer: Absolutely, California does have some cities that might fit what you're looking for. One option could be Santa Barbara. While it's still in California, it tends to have lower crime rates and housing prices compared to some of the more popular cities in the state. Let me know if you have any other questions or preferences, and I can help you narrow down your search even further!


In [ ]:
print(chat_history)

In [ ]:
question = "How is crime rate in Texas?"

answer = generate_answer_main(question, chat_history)

update_history(question, answer)
print("Answer:", answer)


chat_history = summarize_conversation(conversation_history)



In [ ]:
print(chat_history)

In [ ]:
question = "How is the house price in that area?"

answer = generate_answer_main(question, chat_history)

update_history(question, answer)
print("Answer:", answer)

chat_history = summarize_conversation(conversation_history)

In [ ]:
!pip install flask-ngrok pyngrok

In [34]:
!pip install flask_cors

In [ ]:
# This block sets up a Flask API for serving the multi-agent LLM system.
# It handles POST requests at `/chat`, invokes the query handler, and returns friendly, contextual responses.
# Ngrok is used to expose the local Flask server to the internet for quick testing and integration.

from flask import Flask, request, jsonify
from pyngrok import ngrok
import threading
from flask_cors import CORS

# Initialize Flask app and enable CORS
app = Flask(__name__)
CORS(app)

# Basic health check route
@app.route("/")
def home():
    return jsonify({"message": "Flask app is live on ngrok !"})

# Main chat endpoint: receives user question and returns LLM-based answer
@app.route("/chat", methods=["POST"])
def chat():
    data = request.get_json()
    question = data.get("question", "")
        
    update_history(question, answer)

    chat_history = summarize_conversation(conversation_history)

    # Call main multi-agent logic for housing + crime processing and summarization
    answer = generate_answer_main(question, chat_history)

    return jsonify({"reply": answer})

# Run the Flask app in a background thread
def run():
    app.run(port=5004, debug=True, use_reloader=False, threaded=True)

threading.Thread(target=run).start()

# Open a public ngrok tunnel to expose port 5004
public_url = ngrok.connect(5004)
print("Ngrok Tunnel URL:", public_url)

 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5004
INFO:werkzeug:Press CTRL+C to quit


Ngrok Tunnel URL: NgrokTunnel: "https://1d24-34-16-156-139.ngrok-free.app" -> "http://localhost:5004"
